In [1]:
import os
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from operator import add
import numpy as np

In [2]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

MAX_ITER = 20
NUM_CLUSTERS = 10

23/04/26 21:28:24 WARN Utils: Your hostname, X resolves to a loopback address: 127.0.1.1; using 172.30.79.3 instead (on interface eth0)
23/04/26 21:28:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/26 21:28:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))


def load_centroids(file):
    centroids = []
    with open(file, 'r') as f:
        for line in f:
            centroid = parse_line(line)
            centroids.append(centroid)
    centroids = np.array(centroids)
    return centroids


def assign_cluster(point, centroids, distance_metric):
    distances = []
    for centroid in centroids:
        distances.append(distance_metric(point, centroid))
    idx = np.argmin(distances)
    cost = distances[idx]
    return (idx, cost)
        

def parse_line(line):
    point = np.fromstring(line.rstrip(), dtype=np.float32, sep=' ')
    return point

In [19]:
data_path = "data/data.txt"
points = sc.textFile(data_path).map(parse_line).cache()
centriods = load_centroids("data/c1.txt")

In [20]:
# idx: (cost, point, 1)  1 is for counting
assignments = points.map(lambda p: (assign_cluster(p, centriods, euclidean_distance), p))\
.map(lambda it: (it[0][0], (it[0][1], it[1], 1) ))

In [21]:
assignments.collect()

[(0,
  (0.0,
   array([  0.   ,   0.64 ,   0.64 ,   0.   ,   0.32 ,   0.   ,   0.   ,
            0.   ,   0.   ,   0.   ,   0.   ,   0.64 ,   0.   ,   0.   ,
            0.   ,   0.32 ,   0.   ,   1.29 ,   1.93 ,   0.   ,   0.96 ,
            0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
            0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
            0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
            0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
            0.   ,   0.   ,   0.778,   0.   ,   0.   ,   3.756,  61.   ,
          278.   ,   1.   ], dtype=float32),
   1)),
 (1,
  (0.0,
   array([2.100e-01, 2.800e-01, 5.000e-01, 0.000e+00, 1.400e-01, 2.800e-01,
          2.100e-01, 7.000e-02, 0.000e+00, 9.400e-01, 2.100e-01, 7.900e-01,
          6.500e-01, 2.100e-01, 1.400e-01, 1.400e-01, 7.000e-02, 2.800e-01,
          3.470e+00, 0.000e+00, 1.590e+00, 0.000e+00, 4.300e-01, 4.300e-01,
          0.000e+00, 0.000e+00, 0

In [25]:
new_clusters = assignments.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))

In [26]:
new_clusters.collectAsMap()

{0: (61849.17,
  array([6.7529991e+01, 7.8239998e+01, 1.9362009e+02, 1.2926003e+02,
         2.3972995e+02, 7.2160011e+01, 1.1703001e+02, 8.2050011e+01,
         8.3630013e+01, 1.7121004e+02, 5.6690006e+01, 3.4046002e+02,
         6.8160004e+01, 1.7540001e+01, 2.6520004e+01, 1.7196008e+02,
         1.6164001e+02, 1.3885004e+02, 1.0206499e+03, 9.1320000e+01,
         5.7549017e+02, 2.8360992e+02, 1.0379002e+02, 6.6140007e+01,
         2.9264999e+02, 1.0858000e+02, 4.9010006e+01, 3.2980000e+01,
         2.3309999e+01, 2.6629997e+01, 2.2899998e+01, 1.2169998e+01,
         3.2739998e+01, 1.2709998e+01, 2.9969999e+01, 4.0950001e+01,
         8.5269989e+01, 7.5399995e+00, 2.2129997e+01, 4.3900002e+01,
         1.2210001e+01, 2.0990000e+01, 1.9339998e+01, 1.5130001e+01,
         7.3260002e+01, 7.2730003e+01, 3.8900001e+00, 1.5930000e+01,
         6.9033997e+01, 8.5849007e+01, 1.0117000e+01, 1.6062897e+02,
         7.3171974e+01, 5.1770008e+01, 4.0910503e+03, 4.0672000e+04,
         2.0832200e

In [26]:
r = assignments.groupByKey()

In [30]:
r

PythonRDD[26] at RDD at PythonRDD.scala:53